In [1]:
from distutils.command.config import config
import torch
from torch import nn
from torch.backends import cudnn
import matplotlib.pyplot as plt
import numpy as np 
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import time
from dataset import *
from utils import MAPE

@torch.no_grad()
def test_model(model,dataloader,device):
    total_MSE,total_MAPE = 0.,0.
    total_num = 0
    for imgs,targets in dataloader:
        imgs = imgs.to(device)
        targets = targets.to(device).squeeze()
        output = model(imgs).squeeze()
        mse = nn.MSELoss()(output,targets)
        mape = MAPE(targets,output)
        total_MSE += mse * len(imgs)
        total_MAPE += mape * len(imgs)
        total_num += len(imgs)
    return total_MSE / total_num,total_MAPE / total_num

def test_time_displace(dataset_test,window_size,predict_steps):
    targets = []
    for data_item in dataset_test:
        targets.append(data_item[1])
    targets = torch.tensor(targets)
    output = targets.clone()
    output[predict_steps:] = targets[:len(targets)-predict_steps]
    for i in range(predict_steps):
        output[i] = output[0]
    return nn.MSELoss()(output,targets).item()


cudnn.benchmark = True

torch.manual_seed(300)

criterion = nn.MSELoss()
device = torch.device("cuda")

In [2]:
target_map = 1
season = 0
window_size,predict_steps = 6,6

# generate_img(target_map)

start = time.time()
# 23,15
dataset = get_dataset_img([15,10],window_size,predict_steps,[season],debug=False)
# np.save("./dataset/input0/0.npy",dataset)
# dataset = np.load("./dataset/input0/0.npy",allow_pickle=True)

for i in range(len(dataset)):
    # dataset[i][0] = dataset[i][0][5:]
    dataset[i][0] = torch.from_numpy(dataset[i][0]).to(dtype=torch.float)
    dataset[i][1] = torch.from_numpy(dataset[i][1]).to(dtype=torch.float)

cut_pos = int(0.75 * len(dataset))
dataset_train = dataset[:cut_pos]
dataset_test = dataset[cut_pos:]

print(test_time_displace(dataset_test,window_size,predict_steps))

end = time.time()
print("Prepare Data : %f s"%(end-start))

2.681410789489746
Prepare Data : 2.552956 s


In [3]:

# del args
# del models

from config import args,models

model_name = "CNN_Trans"
epoch = 200
batch_size = 64

dataloader_train = DataLoader(dataset_train,batch_size=batch_size,shuffle=True,num_workers=4,pin_memory=True)
dataloader_test = DataLoader(dataset_test,batch_size=batch_size,num_workers=4,pin_memory=True)

model = models[model_name](**args[model_name])
model.to(device)

lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=1e-3)

comment="%s_%d_%d_%d"%(model_name,target_map,season,predict_steps)+"_32_32_wd=2e-3"
writer = SummaryWriter("./tf_dir/%s"%comment,comment=comment)

for i in range(epoch):
    if(i == 10):
        lr *= 0.5
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    if(i == 30):
        lr *= 0.5
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    # if(i == 70):
    #     lr *= 0.1
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = lr

    start = time.time()
    model.train()
    for imgs,targets in dataloader_train:
        imgs = imgs.to(device)
        targets = targets.to(device).squeeze()
        output = model(imgs).squeeze()
        loss = criterion(output,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    
    train_MSE,train_MAPE = test_model(model,dataloader_train,device)
    test_MSE,test_MAPE = test_model(model,dataloader_test,device)

    end = time.time()

    print("Epoch %d : Train MSE : %f, Train MAPE : %f , Test MSE : %f , Test MAPE : %f , Lr : %f , Time: %f s ." % (i,train_MSE,train_MAPE,test_MSE, test_MAPE ,optimizer.param_groups[0]['lr'],end-start))
    writer.add_scalar('train_MSE', train_MSE.item(), i)
    writer.add_scalar('train_MAPE', train_MAPE.item(), i)
    writer.add_scalar('test_MSE', test_MSE.item(), i)
    writer.add_scalar('test_MAPE', test_MAPE.item(), i)

Epoch 0 : Train MSE : 2.961626, Train MAPE : 0.209614 , Test MSE : 3.094241 , Test MAPE : 0.223073 , Lr : 0.001000 , Time: 20.980968 s .
Epoch 1 : Train MSE : 2.489952, Train MAPE : 0.184225 , Test MSE : 2.730361 , Test MAPE : 0.202536 , Lr : 0.001000 , Time: 19.755959 s .
Epoch 2 : Train MSE : 2.068930, Train MAPE : 0.176219 , Test MSE : 2.350615 , Test MAPE : 0.196990 , Lr : 0.001000 , Time: 20.711253 s .
Epoch 3 : Train MSE : 1.859324, Train MAPE : 0.136172 , Test MSE : 2.244945 , Test MAPE : 0.158452 , Lr : 0.001000 , Time: 18.852184 s .
Epoch 4 : Train MSE : 1.811298, Train MAPE : 0.135660 , Test MSE : 2.316382 , Test MAPE : 0.162072 , Lr : 0.001000 , Time: 20.753375 s .
Epoch 5 : Train MSE : 1.433859, Train MAPE : 0.123389 , Test MSE : 1.950968 , Test MAPE : 0.150540 , Lr : 0.001000 , Time: 19.782597 s .
Epoch 6 : Train MSE : 1.566201, Train MAPE : 0.155563 , Test MSE : 2.012789 , Test MAPE : 0.184239 , Lr : 0.001000 , Time: 18.632018 s .
Epoch 7 : Train MSE : 1.612888, Train MAP

KeyboardInterrupt: 